In [1]:
import pandas as pd
import numpy as np
import math
import tensorflow
from sklearn.utils import shuffle
from transformers import T5TokenizerFast, TFT5ForConditionalGeneration



2021-07-12 19:08:24.524232: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-12 19:08:24.524364: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [17]:
data = pd.read_csv('new_dataset.csv',index_col=0)
data=data[['txt','PERSON','NORP','JOB','JOB1','GPE','ORG','TENSE']]
data['TENSE'] = np.where((data['TENSE'] == 'was'),'PAST','PRESENT')
data=shuffle(data)

n = 1000
df_train = data.iloc[:n,1:]
df_label = data.iloc[:n,0:1]


In [3]:
model = TFT5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5TokenizerFast.from_pretrained("t5-small")
optimizer = tensorflow.keras.optimizers.Adam()

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

2021-07-12 19:09:29.598506: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-12 19:09:29.599722: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-12 19:09:29.600231: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (HAL-9000): /proc/driver/nvidia/version does not exist
2021-07-12 19:09:29.608952: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-12 19:09:29.697028: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
batch_size=32
num_of_batches=math.floor(len(df_train)/batch_size)

In [18]:
label=[df_label.values[i][0] for i in range (len(df_label))]
label_encoded=tokenizer.batch_encode_plus(label,padding=True,max_length=900,return_tensors='tf')["input_ids"]

inputbatch=['|'.join(df_train.iloc[i].dropna().values.tolist()) for i in range (len(df_train))]
input_encoded=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=900,return_tensors='tf')["input_ids"]